In [ ]:
from PIL  import ImageGrab as IG
from core import connect   as connect

import pyautogui as     pg
import serial    as     sr
import time      as     tm
import numpy     as     np
import threading as     tr
import os
import csv

In [ ]:
dataFolderName = 'clogData'
busy = False 

if os.path.exists(dataFolderName):
    print('folder exists')
else:
    os.mkdir(dataFolderName)

In [ ]:
# Function to get the current mouse position
def get_mouse_position():
    x, y = pg.position()
    return x, y  # Return the current mouse position

    pylon viewer settings 
    to frame image well
    
    Sensor  Width: 288
    Sensor Height: 375
    Offset      x: 768
    Offset      y: 634

In [ ]:
# framing for image capture

image_width  = 600
image_height = 885
top_left_x   = 526
top_left_y   = 100

expName = 'phi_30v2_ts500_7o5mlMin'
os.mkdir(expName)

def peek(fileNameTime):
    bot_right_x = top_left_x + image_width
    bot_right_y = top_left_y + image_height

    sc1 = IG.grab(bbox=(top_left_x, top_left_y, bot_right_x, bot_right_y))
    imageName = expName+'/' + str(fileNameTime) + '.png'
    sc1.save(imageName)
    sc1.close()

In [ ]:
print("Finding window location")
tm.sleep(0.1)

# Capture top-left corner
print("Hover mouse over top left corner")
tm.sleep(2)
print('Preparing')
tm.sleep(0.5)
tlx, tly = get_mouse_position()  # Update the tlx, tly values
print(f"Mouse position: ({tlx}, {tly})")

tm.sleep(0.1)

# Capture bottom-right corner
print("Hover mouse over bottom right corner")
tm.sleep(2)
print('Preparing')
tm.sleep(0.5)
brx, bry = get_mouse_position()  # Update the brx, bry values
print(f"Mouse position: ({brx}, {bry})")

#bry = tly + 4
# Use the captured positions
test = IG.grab(bbox=(tlx, tly, brx, bry))
test.show()

In [ ]:
global thresholdBrightnesss
global imnum

imnum = 0


scanFolderName = 'scanningFolder'

if os.path.exists(scanFolderName):
    print('folder exists')
else:
    os.mkdir(scanFolderName)

def scan():
    global imnum

    #imnum = imnum + 1
    channelSlice = IG.grab(bbox=(tlx,tly,brx,bry))
    imageName = scanFolderName+'/slice'+str(imnum)+'.png'
    
    channelSlice.save(imageName)
    channelValuesR = list(channelSlice.getdata(band=0))
    channelValuesG = list(channelSlice.getdata(band=1))
    channelValuesB = list(channelSlice.getdata(band=2))
    channelSlice.close()

    # chanValStack = np.vstack([channelValuesR,channelValuesG,channelValuesB])
    # print(chanValStack)
    
    brightness = np.add(np.add(channelValuesR, channelValuesG), channelValuesB) /3 /255
    return brightness
brightness = scan()

thresholdBrightnesss = np.average(brightness) - 0.015

In [ ]:
brightness = scan()

In [ ]:
np.average(brightness)

In [ ]:
np.average(brightness)

In [ ]:
# get open port info
portinfo = connect.getOpenPorts() 

# 'windows' or 'mac'
deviceType = 'windows'

# MUST set baudrate in pump "System Settings", and MUST match this rate:
baudratePUMP = 9600
baudrateINO  = 38400

useArduino = 0

if deviceType == 'mac':
    print('macDevice')
    # initiate Connection object with first open port
    conn = connect.Connection(port = '/dev/tty.usbserial-A10MLXX5', baudrate = baudratePUMP, x = 0, mode = 0,verbose = False)
    
    if useArduino ==1:
        arduino = sr.Serial(port='/dev/tty.usbmodem2101', baudrate=baudrateINO, timeout=1)
        tm.sleep(2)  # Give some time for the connection to establish
elif deviceType == 'windows':
    print('windowsDevice')
    # initiate Connection object with first open port
    conn = connect.Connection(port = 'COM3', baudrate = baudratePUMP, x = 0, mode = 0,verbose = False)
    
    if useArduino ==1:
        arduino = sr.Serial(port='COM7', baudrate=baudrateINO, timeout=1)
        tm.sleep(2)  # Give some time for the connection to establish

In [ ]:
def WDseq(v,r):
    print("withdrawing", v, "mL, at", r,"ml/min")

    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    tm.sleep(0.05)        

    conn.setDiameter(diameter)  
    tm.sleep(0.05)
    conn.setVolume(volume) 
    tm.sleep(0.05)     
    conn.setRate(rate)   
    tm.sleep(0.05)       
    conn.setDelay(delay)  
    tm.sleep(0.05)     
    tm.sleep(1)

    # Start pump
    conn.startPump()

In [ ]:
def INseq(v,r):
    print("infusing", v, "mL, at", r,"ml/min")
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    tm.sleep(0.05)
    conn.setDiameter(diameter) 
    tm.sleep(0.05) 
    conn.setVolume(volume)    
    tm.sleep(0.05)  
    conn.setRate(rate)    
    tm.sleep(0.05)      
    conn.setDelay(delay)    
       
    tm.sleep(1)

    # Start pump
    conn.startPump()

In [ ]:
def unClogSeq(v,r):
    print("begin unclogging")
    global busy
    global running
    
    pump_buffer_time = 2.5
    slowInFactor = 0.5
    flushVolume = 5
    # big infuse
    INseq(v,r)
    tm.sleep(v / r * 60 + pump_buffer_time)

    # cycle 1
    WDseq(flushVolume,r)
    tm.sleep(flushVolume / r * 60 + pump_buffer_time)
    INseq(flushVolume,r * slowInFactor)
    tm.sleep(flushVolume / r / slowInFactor * 60 + pump_buffer_time)

    # # cycle 2
    # WDseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)
    # INseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)

    # # cycle 3
    # WDseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
    # INseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
   
    busy    = False
    running = False

In [ ]:
def unClogSeqImage(v,r):
    print("begin unclogging")
    global busy
    global running
    
    pump_buffer_time = 2.5
    slowInFactor = 1
    flushVolumeIN = 5 + v
    flushVolumeWD = 5 

    # cycle 1
    WDseq(flushVolumeWD,r)
    tm.sleep(flushVolumeWD / r * 60 + pump_buffer_time)
    
    # infuse all

    INseq(flushVolumeIN,r * slowInFactor)
    tm.sleep(flushVolumeIN / r / slowInFactor * 60 + pump_buffer_time)

    # # cycle 2
    # WDseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)
    # INseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)

    # # cycle 3
    # WDseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
    # INseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
   
    busy    = False
    running = False

In [ ]:
def isClog(pressure, clogLim, vinfed):
    global busy
    global t_clog
    global start_time


In [ ]:
def runExp(v,r):
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v               # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    tm.sleep(0.05)
    conn.setDiameter(diameter)  
    tm.sleep(0.05)
    conn.setVolume(volume)      
    tm.sleep(0.05)
    conn.setRate(rate)       
    tm.sleep(0.05)   
    conn.setDelay(delay)     
      

    tm.sleep(1)
    # Start pump
    conn.startPump()

In [ ]:
# def read_arduino():
#     try:
#         arduino.flushInput()
#         data = arduino.readline()#.decode('utf-8').strip()
#         if data and data.isdigit():
#             return int(data)
#         else:
#             print("here")
#             return None
#     except:
#         # Catch any exception and ignore it
#         print("this here")
#         return None

def read_arduino():
    try:
        arduino.flushInput()  # Clear any junk in the input buffer
        data = arduino.readline().decode('utf-8').strip()
        return int(data)
    except:
        return None


In [ ]:
# Function to read from Arduino in a separate thread
def read_arduino_thread():
    global latest_value
    while True:
        value = read_arduino()  # Your function to read from Arduino
        if value is not None:
            latest_value = value
        tm.sleep(0.25)  # Sleep to match Arduino sampling rate

In [ ]:
# Function to read from Arduino in a separate thread
def scan_channel_thread():
    global latest_value_image
    global brightnessTracker
    while True:
        value = scan()  # Your function to read all values in image
        if value is not None:
            latest_value_image = np.average(value)
            brightnessTracker.append(np.average(latest_value_image))
        tm.sleep(0.25)  # Sleep to match Arduino sampling rate


In [ ]:
INseq(14,50)

In [ ]:
#image only system block

global thresholdBrightnesss
global running
global t_clog
global start_time
global latest_value_image
global brightnessTracker

running = False
busy = False

data_list = [0]
time_list = [0]

brightnessTracker = [0]

t_clog = []
latest_value_image = None

exp_run_rate = 7.5 # ml/min
exp_vol      = 45 # ml
start_time   = tm.time()


# Start the Arduino reading thread
threadScan = tr.Thread(target=scan_channel_thread)
threadScan.daemon = True  # Daemon thread will automatically close when the main program exits
threadScan.start()

flowInitializationTime = 2

n      = 10; # The number of past values to store
buffer = np.zeros(n); # Initialize the buffer with zeros
index  = 0; # Index to keep track of the latest entry

print(expName)

# Open CSV files in append mode
dataFileName = 'clogData/test-lightSignal_' + expName + '.csv'
timeFileName = 'clogData/test-clogTimes_' + expName + '.csv'

# Open the CSV files outside the loop
with open(dataFileName, mode='a', newline='') as data_file, \
     open(timeFileName, mode='a', newline='') as time_file:

    data_writer = csv.writer(data_file)
    time_writer = csv.writer(time_file)


try:
    while True:
        # Store the latest value in the buffer
        buffer[index] = latest_value_image

        # Update the index in a circular manner
        index = (index + 1) % n
        
        avgBrightness = np.mean(buffer)

        tm.sleep(0.1)
        if avgBrightness is not None:
            if not running and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)
                tm.sleep(flowInitializationTime)


            if avgBrightness >= thresholdBrightnesss and not busy:
                t_clog     = tm.time() - start_time_clog
                round_time = np.round(tm.time() - start_time, 0)
 
                vinfed = exp_run_rate / 60 * t_clog
                print("value rose to ", avgBrightness, "which is above brightness limit of" ,thresholdBrightnesss )
                print("Time to clog is ", t_clog)
                busy = True
                
                peek(round_time) # take image of clog
                
                unClogSeqImage(v=vinfed, r=50)

                data_list.append(avgBrightness)
                time_list.append(t_clog)
                
                # Append to CSV files
                data_writer.writerow([avgBrightness])
                time_writer.writerow([t_clog])


except KeyboardInterrupt:
    print("Exiting...")
    conn.stopPump()
    # save data
    dataFileName = 'clogData/lightSignal_' + expName
    timeFileName = 'clogData/clogTimes_' + expName 
    np.savetxt(timeFileName, time_list, delimiter = ',', fmt= '%f')
    print(f"clogging time list saved to {timeFileName}")
    np.savetxt(dataFileName, brightnessTracker, delimiter = ',', fmt= '%f')
    print(f"brightness tracker data saved to {dataFileName}")
    


print("Data collection complete.")


In [ ]:
WDseq(5,5)

In [ ]:
conn.stopPump()

In [ ]:
INseq(2,20)

In [ ]:
import threading
import time as tm
import numpy as np

global running
global t_clog
global start_time
global latest_value

running = False
busy = False

data_list = []
time_list = [0]
pressureOfClog = 93 

t_clog = []
latest_value = None

exp_run_rate = 0.5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()


# Start the Arduino reading thread
thread = threading.Thread(target=read_arduino_thread)
thread.daemon = True  # Daemon thread will automatically close when the main program exits
thread.start()

print(expName)
try:
    while True:
        value = latest_value
         # print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        
        if value is not None and value > 85:
            if not running and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time() - start_time
            P = value
            cL = pressureOfClog

            if P <= cL and not busy:
                t_clog     = tm.time() - start_time_clog
                round_time = np.round(tm.time() - start_time, 0)
 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                
                peek(round_time) # take image of clog
                
                unClogSeq(v=vinfed, r=50)

                time_list.append(t_clog)


except KeyboardInterrupt:
    print("Exiting...")
    conn.stopPump()
    


print("Data collection complete.")


In [ ]:
global running
global t_clog
global start_time

running = False

data_list = []
time_list = []
pressureOfClog = 90 

t_clog = []

exp_run_rate = 5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()

try:
    while True:
        value = read_arduino()
        print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        if value is not None and value > 85:
            if running is not True and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time()-start_time
            #time_list.append(time_running)
            #data_list.append(value)
            #pressmBar = (value - 100) * 2000/398
            

            # isClog(pressure = value, clogLim = pressureOfClog, vinfed= volume)
            P = value
            cL = pressureOfClog

            # print("pressure is ", P)
            # print("clog lim is ", cL)
            if P <= cL and not busy:
                t_clog = tm.time() - start_time_clog 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                unClogSeq(v=vinfed,r=50)

            # Print the latest value every few seconds (e.g., every 5 seconds)
            round_time = np.round(tm.time() - start_time,0)
            # if round_time % 3 == 0:  # 0.1s * 30 = 3 seconds
            #     print(f"Latest value (after {round_time} seconds): {value}")
            #     # print("press mbar is", pressmBar)
        
except KeyboardInterrupt:
    print("Exiting...")
    # Handle any cleanup if necessary here

print("Data collection complete.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Create a time vector with 0.25s intervals
# Assuming y has length N, time vector will have N points
N = len(brightnessTracker)
t = np.arange(0, N * 0.25, 0.25)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(t, brightnessTracker, marker='o', linestyle='-', color='b')
plt.xlabel('Time (s)')
#plt.xlim([0,600])
plt.ylabel('channel brightness')
plt.title('clogging visualization')
plt.grid(True)
plt.show()
